In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import time
import pickle

In [7]:
opts=Options()
opts.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36')

In [8]:
links=[['https://origendirecto.com/ecommerce/#/items?category=Aderezos__Aderezos','aderezos'],
      ['https://origendirecto.com/ecommerce/#/items?category=Almacen__Almacen','almacen'],
      ['https://origendirecto.com/ecommerce/#/items?category=Frutas%2FVerduras__Frutas%2FVerduras','verduras'],
      ['https://origendirecto.com/ecommerce/#/items?category=Hamburguesas__Hamburguesas','hamburguesas'],
      ['https://origendirecto.com/ecommerce/#/items?category=Infusiones__Infusiones','infusiones'],
      ['https://origendirecto.com/ecommerce/#/items?category=Lacteos__Lacteos','lacteos'],
      ['https://origendirecto.com/ecommerce/#/items?category=Milanesas__Milanesas','milanesas'],
      ['https://origendirecto.com/ecommerce/#/items?category=Panificados__Panificados','panificados'],
      ['https://origendirecto.com/ecommerce/#/items?category=Papas__Papas','papas'],
      ['https://origendirecto.com/ecommerce/#/items?category=Pastas__Pastas','pastas'],
      ['https://origendirecto.com/ecommerce/#/items?category=Postres%2FGolosinas__Postres%2FGolosinas','golosinas'],
      ['https://origendirecto.com/ecommerce/#/items?category=QuesosyFiambres__Quesos%2FFiambres%2FSalchichas','queso/fiambres/salchichas']]

In [9]:
driver=webdriver.Chrome(chrome_options=opts)
catalogo={}
precios_final=[]
productos_final=[]
categoria_concat=[]

for categoria in links:
    driver.get(categoria[0])
    driver.refresh()
    while True:
        time.sleep(8)
        
        productos=driver.find_elements_by_xpath('//p[@class="card-text item-name"]') #Nombre
        precios=driver.find_elements_by_xpath('//p[@class="card-text item-price mb-0"]') #Precio
        for precio in precios:
            posicion_uni=precio.text.find('UNI')
            precios_final.append(precio.text[2:posicion_uni])
        for producto in productos:
            productos_final.append(producto.text)
            categoria_concat.append(categoria[1])

        boton=driver.find_elements_by_xpath('//a[@class="page-link"]/span[text()="»"]')
        try:
            boton[1].click()
            time.sleep(20)
        except:
            break;

for i in range(len(productos_final)):
    catalogo[productos_final[i]]=[categoria_concat[i],precios_final[i]]
    
df_catalogo = pd.DataFrame(zip(productos_final,categoria_concat,precios_final),columns=['productos','categoria',datetime.today().strftime('%Y-%m-%d')])

<ipython-input-9-8a93b25aa393>:1: DeprecationWarning: use options instead of chrome_options
  driver=webdriver.Chrome(chrome_options=opts)


In [10]:
with open('historial_precios.pkl', 'rb') as f_math:
    df_historico = pickle.load(f_math)

In [11]:
historico_actualizado=pd.concat([df_historico,df_catalogo[['productos','categoria']]])
historico_actualizado.drop_duplicates(subset ="productos", keep = 'first', inplace = True)

In [12]:
df_historico=pd.merge(historico_actualizado, df_catalogo[['productos',datetime.today().strftime('%Y-%m-%d')]], how='left', on='productos')
df_historico.sample(3)

,productos,categoria,2020-08-12,2020-08-13,2020-09-03,2020-09-29
131,MILANESA CERDO UNION GANADERA 700 gr,milanesas,290.00,290.00,290.00,NaN
240,YOGURT ENTERO FRUTILLA TREGAR X125GR. S/TACC,lacteos,NaN,NaN,21.00,21.00
133,MILANESA PECETO UNION GANADERA 500 gr,milanesas,265.00,265.00,265.00,265.00


In [13]:
with open('historial_precios.pkl', 'wb') as f_math: #wb: permiso
    pickle.dump(df_historico, f_math) #f_math es una manera, mientras coincida con el de arriba esta optimo